Checking origional data from: https://data.mendeley.com/datasets/rscbjbr9sj/2

In [1]:
#Imports
from wrangle import *
import numpy as np
import pandas as pd
import os, shutil, glob, itertools, random
import matplotlib.pyplot as plt
import warnings
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner
from keras_tuner import RandomSearch, GridSearch
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image

In [2]:
#Preferences
warnings.simplefilter(action="ignore", category=FutureWarning)
%matplotlib inline

In [3]:
if not os.path.exists("data/chest_xray"):
    acquire_data()

Starting download...
Download Complete
Extracting files
Files extracted


In [4]:
if not os.path.exists("data/chest_xray/val"):
    prepare_data()

In [5]:
save_model = True
save_weights = True

In [6]:
#Path variables
train_path = "data/train/"
val_path = "data/val/"
test_path = "data/test/"

In [7]:
path_list = [train_path, val_path, test_path]

In [8]:
datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = datagen.flow_from_directory(directory= train_path,
                                              target_size=(256,256),
                                              batch_size=16,
                                              color_mode="grayscale",
                                              classes=["normal", "viral_pneumonia", "bacterial_pneumonia"],
                                              save_format="jpeg")

Found 4332 images belonging to 3 classes.


In [10]:
val_generator = datagen.flow_from_directory(directory=val_path,
                                            target_size=(256,256),
                                            batch_size=16,
                                            color_mode="grayscale",
                                            classes=["normal", "viral_pneumonia", "bacterial_pneumonia"],
                                            save_format="jpeg")

Found 900 images belonging to 3 classes.


In [11]:
test_generator = datagen.flow_from_directory(directory=test_path,
                                             target_size=(256,256),
                                             batch_size=16,
                                             color_mode="grayscale",
                                             classes=["normal", "viral_pneumonia", "bacterial_pneumonia"],
                                             save_format="jpeg",
                                             shuffle=False)

Found 624 images belonging to 3 classes.


In [12]:
img, labels = next(train_generator)

In [13]:
def build_model(hp):
    """
    Building a model to optimize hypter parameters.
    """
    model = Sequential()
    model.add(Conv2D(filters=hp.Int("conv_1_filter",
                                    min_value=8,
                                    max_value=128,
                                    step=16),
                     kernel_size=hp.Choice("conv_1_kernel", values = [3,5]),
                     activation="relu",
                     input_shape=(256,256,1)))
    model.add(MaxPool2D(pool_size=2))
    model.add(Conv2D(filters=hp.Int("conv_2_filter",
                                    min_value=8,
                                    max_value=128,
                                    step=16),
                     kernel_size=hp.Choice("conv_2_kernel", values = [3,5]),
                     activation="relu"))
    model.add(MaxPool2D(pool_size=2))
    model.add(Conv2D(filters=hp.Int("conv_3_filter",
                                    min_value=8,
                                    max_value=128,
                                    step=16),
                     kernel_size=hp.Choice("conv_3_kernel", values = [3,5]),
                     activation = "relu"))
    model.add(MaxPool2D(pool_size=2))
    model.add(Conv2D(filters=hp.Int("conv_4_filter",
                                    min_value=8,
                                    max_value=128,
                                    step=16),
                     kernel_size=hp.Choice("conv_4_kernel", values = [3,5]),
                     activation="relu",
                     input_shape=(256,256,1)))
    model.add(MaxPool2D(pool_size=2))
    model.add(Conv2D(filters=hp.Int("conv_5_filter",
                                    min_value=8,
                                    max_value=128,
                                    step=16),
                     kernel_size=hp.Choice("conv_5_kernel", values = [3,5]),
                     activation="relu",
                     input_shape=(256,256,1)))
    model.add(MaxPool2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=hp.Int("dense_1_units",
                                 min_value=8,
                                 max_value=8192,
                                 step=256),
                    activation="relu", kernel_regularizer=l2(hp.Float("l2_value", min_value=1e-5, max_value=1e-2, sampling="LOG"))))
    model.add(Dropout(hp.Float("dropout_1", min_value=0.0,max_value=0.8, step=0.1)))
    model.add(Dense(units=hp.Int("dense_2_units",
                                 min_value=8,
                                 max_value=4096,
                                 step=128),
                    activation="relu", kernel_regularizer=l2(hp.Float("l2_value", min_value=1e-5, max_value=1e-2, sampling="LOG"))))
    model.add(Dense(units=3, activation="softmax"))
    model.compile(optimizer=Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3])),
                  loss="categorical_crossentropy",
                  metrics=["accuracy", tf.keras.metrics.Recall()])

    return model

In [14]:
earlystop = EarlyStopping(monitor="val_loss", patience = 4)

In [15]:
tuner_search = GridSearch(build_model,
                          objective="val_accuracy",
                          max_trials=20,
                          executions_per_trial=3,
                          directory="TriclassOutput",
                          project_name="ChestXrayTriClass0")

2023-08-10 09:01:17.056691: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
tuner_search.search(train_generator, epochs=3, validation_data=val_generator)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
8                 |8                 |conv_1_filter
3                 |3                 |conv_1_kernel
16                |16                |conv_2_filter
3                 |3                 |conv_2_kernel
16                |16                |conv_3_filter
3                 |3                 |conv_3_kernel
16                |16                |dense_1_units
1e-05             |1e-05             |l2_value
0.01              |0.01              |learning_rate

Epoch 1/3
271/271 [==============================] - 224s 826ms/step - loss: 0.9356 - accuracy: 0.6060 - recall: 0.4778 - val_loss: 0.8931 - val_accuracy: 0.5700 - val_recall: 0.4456
Epoch 2/3
271/271 [==============================] - 220s 809ms/step - loss: 0.7637 - accuracy: 0.6745 - recall: 0.6062 - val_loss: 0.8617 - val_accuracy: 0.5911 - val_recall: 0.4822
Epoch 3/3
271/271 [==============================] - 221s 815ms/step - loss: 0.7168 - accu

In [ ]:
tuner_search.search_space_summary()

In [ ]:
tuner_search.results_summary()

In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[earlystop])

### Make a prediction

In [ ]:
#Plot function
def plotImages(images_arr):
    """
    Plots images in a gird.
    """
    fig, axes, = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img, cmap='gray')
        ax.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
test_imgs, test_labels = next(test_generator)
plotImages(test_imgs)
print(test_labels)

In [ ]:
predictions = model.predict(x=test_generator, verbose=0)

In [ ]:
cm = confusion_matrix(y_true=test_generator.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
#Confusion matrix plot function from TensorFlows website
def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues):
    """
    A function to plot results in a confusion matrix.
    """
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i,j], horizontalalignment="center", color = "white" if cm[i,j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
cm_plot_labels = ["Normal","viral_pneumonia", "bacterial_pneumonia"]
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title = "Confusion Matrix")

In [ ]:
#Saving the model
if save_model == True:
    model.save("models/pneumonia_detection.h5")
    
if save_weights == True:
    model.save_weights("models/pneumonia_detection_weights.h5")

In [ ]:
confusion_matrix(y_true=test_generator.classes, y_pred=np.argmax(predictions, axis=-1), normalize="all")

In [ ]:
print(classification_report(y_true=test_generator.classes, y_pred=np.argmax(predictions, axis=-1), target_names = cm_plot_labels))

In [ ]:
# Plot Training and Validation Accuracy
plt.figure(figsize=(14,4))
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Model Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

# Plot Training and Validation Loss
plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()